In [4]:
#packages
%pip install pandas gensim string

import pandas as pd 
import gensim.downloader as api
import argparse
import string

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Load in data

In [5]:
song_data = pd.read_csv("../data/Spotify Million Song Dataset_exported.csv")

In [6]:
song_data

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


# Load in gensim model

In [7]:
model = api.load("glove-wiki-gigaword-50")

# Function returning the percentage of a given artist's songs that contain the words related to search word

In [ ]:
def get_artist_and_word():
    parser= argparse.ArgumentParser(description= "Loading and printing an array")
    parser.add_argument("--artist", 
                        "-a",
                        required= True, #must give it an input in the command 
                        help= "put name of artist") #help message if input is missing 
    parser.add_argument("--search_term", #long form name
                        "-s",#short form name,change
                        required= True, #must give it an input in the command 
                        help= "put a search term") #help message if input is missing 
    args = parser.parse_args()
    args.artist= args.artist.lower()
    args.search_term= args.search_term.lower()
    return args

In [ ]:
def calculate_stats(args): 
    name_of_artist = args.artist
    chosen_search_term = args.search_term

    model = api.load("glove-wiki-gigaword-50")
    song_data = pd.read_csv("data/Spotify Million Song Dataset_exported.csv")

    similar_words_list = []
    words = model.most_similar(chosen_search_term)
    for word, _ in words:
        similar_words_list.append(word)

    # Remove punctuation from the 'text' column
    song_data['text'] = song_data['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
        
    artist_df = song_data[song_data['artist'].str.lower() == name_of_artist]

    songs_already_counted = set()  # Set to keep track of songs already counted

    n_songs_where_word_from_wordslist_appears = 0
    for word in similar_words_list:
        # Iterate over each song's text
        for song_text in artist_df['text'].str.lower():
            # Check if the word appears in the song and if the song hasn't been counted yet
            if word in song_text and song_text not in songs_already_counted:
                n_songs_where_word_from_wordslist_appears += 1
                songs_already_counted.add(song_text)  # Add the song to the set to mark it as counted

    total_n_songs_for_artist = len(artist_df)

    percentage = round(((n_songs_where_word_from_wordslist_appears / total_n_songs_for_artist) * 100),2)

    return name_of_artist, chosen_search_term, percentage

In [ ]:
def main():
    #call args fuction 
    args= get_artist_and_word()
    #function for calculation stuff
    name_of_arist, chosen_search_term, percentage= calculate_stats(args)
    print(f"{percentage}% of {name_of_arist}'s songs contain words related to {chosen_search_term}")
    


# check by doing something like

In [37]:
name_of_arist

'Taylor Swift'

In [38]:
chosen_search_term

'dog'

In [39]:
similar_words_list

['cat',
 'dogs',
 'horse',
 'puppy',
 'pet',
 'rabbit',
 'pig',
 'snake',
 'baby',
 'bite']

In [40]:
artist_df

,artist,song,link,text
19137,Taylor Swift,Cold As You,/t/taylor+swift/cold+as+you_20359713.html,You have a way of coming easily to me \nAnd w...
19138,Taylor Swift,Crazier,/t/taylor+swift/crazier_20783496.html,"I've never gone with the wind, just let it flo..."
19139,Taylor Swift,Everything Has Changed,/t/taylor+swift/everything+has+changed_2106343...,All I knew this morning when I woke \nIs I kn...
19140,Taylor Swift,Fifteen,/t/taylor+swift/fifteen_20766168.html,You take a deep breath \nAnd you walk through...
19141,Taylor Swift,Hey Stephen,/t/taylor+swift/hey+stephen_20758965.html,"Hey Stephen, I know looks can be deceiving \n..."
...,...,...,...,...
52781,Taylor Swift,Tim McGraw,/t/taylor+swift/tim+mcgraw_20359673.html,He said the way my blue eyes shined \nPut tho...
52782,Taylor Swift,Today Was A Fairytale,/t/taylor+swift/today+was+a+fairytale_20761546...,Today was a fairytale \nYou were the prince ...
52783,Taylor Swift,White Christmas,/t/taylor+swift/white+christmas_20658358.html,I'm dreaming of a white Christmas \nJust like...
52784,Taylor Swift,White Horse,/t/taylor+swift/white+horse_20761227.html,Say you're sorry \nThat face of an angel \nC...


In [41]:
total_n_song_for_artist

81

In [42]:
n_songs_where_word_from_wordslist_appears

26

In [43]:
percentage

32.098765432098766